Ставим Efficientnet

In [ ]:
!pip install -q efficientnet

Импорт библиотек

In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn import metrics
from sklearn.model_selection import train_test_split

## ______________ БЛОК С ИМПОРТАМИ АРХИТЕКТУР ____________________
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201 
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2
from tensorflow.keras.applications.nasnet import NASNetLarge
from efficientnet.tfkeras import EfficientNetB7, EfficientNetL2
## ______________ КОНЕЦ БЛОКА С ИМПОРТАМИ АРХИТЕКТУР ____________________

# импорт других полезных инструментов: слоев, оптимизаторов, функций обратной связи
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

## Настраиваем TPU конфигурацию

In [ ]:
# Путь к данным. Если работаете на Google Colaboratory, то замените KaggleDatasets().get_gcs_path() на путь к данным, который будет у вас
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Проверяем существующее оборудование и выбираем соответствующую стратегию использования вычислений
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    # онаружение TPU. Параметры не требуются если установленна переменная окружения TPU_NAME. На Kaggle это всегда True.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # если TPU отсутствует, то испльзуем стратегию по умолчанию для TF (CPU or GPU)

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Конфигурация
EPOCHS = 35
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
NB_CLASSES = 4
STEPS_PER_EPOCH = 1821 // BATCH_SIZE
print(STEPS_PER_EPOCH)

Посмотрим на данные

## Загружаем метки классов и пути к изображениям

In [ ]:
# функция, которая превращает айди картинки в полный путь к ней
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')
sub = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')

train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values

train_labels = train.loc[:, 'healthy':].values

# если планируете обучать модель с валидирующим набором данных
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train_paths, train_labels, test_size=0.15, random_state=2020)

In [ ]:
from matplotlib import pyplot as plt

# создаем сетку 2 на 5, для более компактного отображения символов и задаем размер их отображения
f, ax = plt.subplots(3, 6, figsize=(18, 7))
ax = ax.flatten()
# отрисовываем в цикле найденные топ N изображений частей графем
for i in range(18):
    img = plt.imread(f'../input/plant-pathology-2020-fgvc7/images/Train_{i}.jpg')
    ax[i].set_title(train[train['image_id']==f'Train_{i}'].melt()[train[train['image_id']==f'Train_{i}'].melt().value == 1]['variable'].values[0])
    ax[i].imshow(img)
print(img.shape)

## Создаем объекты наборов данных

A `tf.data.Dataset` объект нужен для того, чтобы модель бесперебойно работала на TPUs

In [ ]:
# устанавливаем глобальные переменные
img_size = 728

# функция, которая читает изображение из файла и преобразовывает его к нужному размеру, а так же нормализует
def decode_image(filename, label=None, image_size=(img_size, img_size)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label
# функция расширения данных

Создаем объекты наборов данных для обучения, валидации и теста

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
#     .map(data_augment, num_parallel_calls=AUTO) # если напишите свою функцию расширения данных data_augment 
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
# если планируете обучать модель с валидирующим набором данных
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

## Строим и обучаем модель

### Вспомогательные функции

In [ ]:
def get_model(use_model):
    base_model = use_model(
        weights='imagenet', 
        include_top=False, pooling='avg',
        input_shape=(img_size, img_size, 3))
    
    #base_model.trainable = False
    
    x = Flatten()(base_model.layers[-1].output)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)

    #x = Dropout(0.4)(x)
    predictions = Dense(NB_CLASSES, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)

### Загружаем модель на TPU

In [ ]:
with strategy.scope():    
    model = get_model(EfficientNetB7)
    
model.compile(
    optimizer='nadam',
    loss = 'categorical_crossentropy',
    metrics=['categorical_accuracy'])

### Запускаем процесс обучения

In [ ]:
history = model.fit(train_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, restore_best_weights=True),
                     ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1),
                     ModelCheckpoint(filepath='my_ef_net_b7.h5', monitor='val_categorical_accuracy', save_best_only=True)],
          validation_data= valid_dataset,
          workers = 3)

## Проверка качества модели

Unhide below to see helper function `display_training_curves`:

In [ ]:
plt.plot(history.history['categorical_accuracy'], 
         label='Оценка точности на обучающем наборе')
plt.plot(history.history['val_categorical_accuracy'], 
         label='Оценка точности на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Оценка точности')
plt.legend()
plt.show()

## Делаем прогноз тестовых данных и готовим представление данных для проверки

Когда вы получите несколько моделей с высокими баллами, можно поробовать их объеденить

probs = (model1.predict(test_dataset)+model.predict(test_dataset))/2

где можно использовать и больше моделей, но тогда делить не на 2, а на количество использованных моделей

In [ ]:
probs = model.predict(test_dataset)
sub.loc[:, 'healthy':] = probs
sub.to_csv('submission.csv', index=False)
sub.head()

In [ ]:
# создаем сетку 2 на 5, для более компактного отображения символов и задаем размер их отображения
f, ax = plt.subplots(3, 5, figsize=(18, 8))
ax = ax.flatten()
# отрисовываем в цикле найденные топ N изображений частей графем
for i in range(10):
    img = plt.imread(f'../input/plant-pathology-2020-fgvc7/images/Test_{i}.jpg')
    ax[i].set_title(sub[sub['image_id']==f'Test_{i}'].melt().iloc[1:][sub[sub['image_id']==f'Test_{i}'].melt().iloc[1:].value >= 0.8]['variable'].values[0])
    ax[i].imshow(img)
print(img.shape)